In [3]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [4]:
#Data - Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
            result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
        return result

In [5]:
#Data - Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Data - Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [6]:
#Data - Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [7]:
#Data - Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [8]:
#Data - Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [9]:
#Data - Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [10]:
#Data - Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [11]:
#Data - Train the model
model.fit(x_train,y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_iter=500, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [12]:
#Data- Predict for the test set
y_pred=model.predict(x_test)

In [13]:
print(len(x_test))

192


In [14]:
print(x_test)

[[-6.01416077e+02  6.33363037e+01 -1.16155634e+01 ...  7.30560214e-06
   4.64440200e-06  1.78248013e-06]
 [-7.87200195e+02  5.97919426e+01  2.11022396e+01 ...  4.30602057e-07
   4.01216226e-07  4.06060309e-07]
 [-5.60616211e+02  5.78973503e+01 -8.86894608e+00 ...  6.46989676e-04
   3.31015413e-04  1.85833647e-04]
 ...
 [-4.44761963e+02  2.36135769e+01 -1.48351011e+01 ...  8.65290698e-04
   5.16015454e-04  3.73600313e-04]
 [-5.96088257e+02  6.75587769e+01  1.62860453e+00 ...  2.92377263e-05
   2.20639995e-05  1.13877868e-05]
 [-6.95250610e+02  4.46737938e+01 -7.46818161e+00 ...  2.15334803e-05
   1.84785295e-05  1.01300748e-05]]


In [15]:
print(y_pred)

['happy' 'calm' 'happy' 'happy' 'disgust' 'calm' 'calm' 'happy' 'calm'
 'happy' 'happy' 'calm' 'fearful' 'happy' 'disgust' 'happy' 'calm'
 'disgust' 'happy' 'calm' 'disgust' 'disgust' 'disgust' 'calm' 'happy'
 'happy' 'disgust' 'happy' 'calm' 'happy' 'happy' 'happy' 'happy' 'calm'
 'happy' 'calm' 'calm' 'fearful' 'calm' 'disgust' 'happy' 'disgust' 'calm'
 'calm' 'happy' 'disgust' 'disgust' 'calm' 'happy' 'happy' 'fearful'
 'fearful' 'fearful' 'happy' 'happy' 'calm' 'disgust' 'happy' 'calm'
 'calm' 'disgust' 'calm' 'happy' 'happy' 'disgust' 'calm' 'calm' 'calm'
 'disgust' 'happy' 'fearful' 'fearful' 'fearful' 'fearful' 'fearful'
 'disgust' 'happy' 'happy' 'calm' 'fearful' 'disgust' 'calm' 'happy'
 'calm' 'disgust' 'happy' 'disgust' 'happy' 'happy' 'happy' 'disgust'
 'happy' 'calm' 'happy' 'disgust' 'disgust' 'calm' 'calm' 'calm' 'disgust'
 'fearful' 'disgust' 'disgust' 'fearful' 'disgust' 'calm' 'disgust'
 'happy' 'fearful' 'happy' 'happy' 'calm' 'calm' 'fearful' 'fearful'
 'calm' 'calm

In [16]:
from joblib import dump, load
dump(model, 'ser.joblib') 

['ser.joblib']

In [17]:
print(x_test)

[[-6.01416077e+02  6.33363037e+01 -1.16155634e+01 ...  7.30560214e-06
   4.64440200e-06  1.78248013e-06]
 [-7.87200195e+02  5.97919426e+01  2.11022396e+01 ...  4.30602057e-07
   4.01216226e-07  4.06060309e-07]
 [-5.60616211e+02  5.78973503e+01 -8.86894608e+00 ...  6.46989676e-04
   3.31015413e-04  1.85833647e-04]
 ...
 [-4.44761963e+02  2.36135769e+01 -1.48351011e+01 ...  8.65290698e-04
   5.16015454e-04  3.73600313e-04]
 [-5.96088257e+02  6.75587769e+01  1.62860453e+00 ...  2.92377263e-05
   2.20639995e-05  1.13877868e-05]
 [-6.95250610e+02  4.46737938e+01 -7.46818161e+00 ...  2.15334803e-05
   1.84785295e-05  1.01300748e-05]]


In [18]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 71.88%


# Checking the output

In [19]:
        x,y=[],[]
        file_name=os.path.basename("Actor_01\\03-01-02-02-02-02-01.wav")
        
        feature1=extract_feature("Actor_01\\03-01-02-02-02-02-01.wav", mfcc=True, chroma=True, mel=True)
        x.append(feature1)
        



In [20]:
from array import array
ar=np.asarray(feature1)

ar=ar.reshape(1,-1)
emotion=model.predict(ar)
print(emotion)


['happy']


# Giving file path as input

In [22]:
ch='y'
while ch=='y':

    xyz=str(input("wav"))
    file_name=os.path.basename(xyz)
    feature1=extract_feature(xyz,mfcc=True,chroma=True,mel=True)
    x.append(feature1)
    
    from array import array
    ar=np.asarray(feature1)

    ar=ar.reshape(1,-1)
    emotion=model.predict(ar)
    print(emotion)
    ch=input("enter ch== y or n")

    




wavActor_01\\03-01-02-02-02-02-01.wav
['happy']
enter ch== y or nn


# Browsing for WAV files and giving as input

In [ ]:
from tkinter import * 

from tkinter import filedialog

import sys
if sys.version_info[0] < 3:
   import Tkinter as Tk
else:
   import tkinter as Tk


def browse_file():

    fname = filedialog.askopenfilename(filetypes = (("wav files", "*.wav"), ("All files", "*")))
    print(fname)
    xyz=fname
    file_name=os.path.basename(xyz)
    feature1=extract_feature(xyz,mfcc=True,chroma=True,mel=True)
    x.append(feature1)
    
    from array import array
    ar=np.asarray(feature1)

    ar=ar.reshape(1,-1)
    emotion=model.predict(ar)
    print(emotion)

root = Tk.Tk()
root.wm_title("INPUT FOR SPEECH EMOTION RECOGNITION")
broButton = Tk.Button(master = root, text = 'Browse for file to be given as input', width = 50,command=browse_file)
broButton.pack(side=Tk.TOP, padx = 500, pady=2)

Button(root, text="Exit from recognition",width = 30, command=root.destroy).pack()



Tk.mainloop()


C:/Users/rachanarshini/OneDrive/Desktop/ser/Actor_07/03-01-08-02-02-02-07.wav
['disgust']
